Primera versión con libreria propia:

In [22]:
# DEFINICION DE LAS RUTAS Y VARIABLES:

# Carpeta donde se encuentran las auxiliares:
ruta_aux = 'C:/modelos/auxiliares/'
aux_pmo = 'Aux_PMO.xlsx'
medicamentos_especiales = ruta_aux + 'Aux_Prestaciones_especiales_med.xlsx'
cobertura_medicamentos = ruta_aux + 'Aux_CobertMedicamentosPlan.xlsx'

# Detalle con el perfil de cada asociado:
archivo_perfil_asociados = 'C:/Users/lsaucedo/downloads/resumen_asociados.csv'

# Ruta con Consumos de la zona:
ruta_consumos_zona = 'C:/Users/lsaucedo/Desktop/Costo PMO 202112/Consumos_Zona/'

# Ruta con Stock de la zona:
ruta_stock_zona = 'C:/Users/lsaucedo/Desktop/Costo PMO 202112/Stocks_Zona/'


ruta_int = 'C:\\modelos\\Internaciones\\50.1 Tipos de internaciones.xlsx' # tipos de internaciones
ruta_diag = 'C:/Users/lsaucedo/downloads/diagnostico.csv'


## PARAMETROS OBLIGATORIOS:
resultado_consumos = 'C:/Users/lsaucedo/downloads/consumo_pmo_zona.csv'
resultado_informe = 'C:/Users/lsaucedo/downloads/informe_pmo_zona.csv'

In [8]:
# Librerías usadas:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import funciones

## Levantar los archivos de consumos

In [9]:
# Generamos dataframe de consumos:
dfc = funciones.leer_xlsx(ruta = ruta_consumos_zona)

Inicio del proceso: 16:10:07
Se cargó el archivo:  10.1 Cons Mendoza 202010.xlsx a las: 16:11:04
Se cargó el archivo:  10.1 Cons Mendoza 202011.xlsx a las: 16:12:08
Se cargó el archivo:  10.1 Cons Mendoza 202012.xlsx a las: 16:13:10
Se cargó el archivo:  10.1 Cons Mendoza 202101.xlsx a las: 16:14:16
Se cargó el archivo:  10.1 Cons Mendoza 202102.xlsx a las: 16:15:19
Se cargó el archivo:  10.1 Cons Mendoza 202103.xlsx a las: 16:16:40
Se cargó el archivo:  10.1 Cons Mendoza 202104.xlsx a las: 16:18:06
Se cargó el archivo:  10.1 Cons Mendoza 202105.xlsx a las: 16:19:28
Se cargó el archivo:  10.1 Cons Mendoza 202106.xlsx a las: 16:20:53
Se cargó el archivo:  10.1 Cons Mendoza 202107.xlsx a las: 16:22:21
Se cargó el archivo:  10.1 Cons Mendoza 202108.xlsx a las: 16:23:48
Se cargó el archivo:  10.1 Cons Mendoza 202109.xlsx a las: 16:25:19
Fin de la carga de archivos


In [10]:
# renombramos las columnas:
funciones.renombrar_columnas(df = dfc)

In [11]:
# CREAMOS LA COLUMNA: "Marca PMO"

# Levanto el excel con marca de prestaciones pmo
dfpmo = pd.read_excel(ruta_aux + 'Aux_pmo.xlsx')
dfc["Prest_sin_esp"] = dfc.Prestacion.str.strip() # Quito los espacios a las prestaciones:
# Unimos la marca al dfc
dfc = pd.merge( left = dfc, right = dfpmo, left_on='Prest_sin_esp', right_on='Prest sin espacios', how = "left")
del dfpmo # Libero memoria:
print('Las prestaciones que no se pudieron catalogar fueron:') # Prestaciones detectadas como faltantes:
print(dfc[dfc['Marca PMO'].isna()]["Prestacion"].unique()) # Prestaciones detectadas como faltantes:
dfc['Marca PMO'] = dfc['Marca PMO'].replace(np.nan,"NO PMO") # Reemplazamos los nan de marca pmo por "NO PMO"

Las prestaciones que no se pudieron catalogar fueron:
['NS100     ' 'NS0405710 ' 'NS0401050 ' 'NS0401140 ' 'NS0401080 '
 'NS0404020 ' 'NS0404100 ' 'NS0401110 ' 'NS0404110 ' 'NS0401120 '
 'NS0402040 ' '00330101  ' '00342089  ']


In [12]:
## SEGMENTACIONES Y METRICAS POSIBLES ANTES DE EXPORTAR:

# Segmentaciones y métricas comúnes:
segmentacion = (['Periodo_Prest', 'Persona'])

# dejo para probar procesamientos más rápido (debugging)
segmentacion2 = ['Periodo', 'Sexo']


# Defino las métricas a calcular
metricas = ({'Consumo':[np.sum], 'Prestacion':pd.Series.nunique,'Cantidad':[np.sum]})


metricas2 = ({  'Consumo':[np.sum,len],
                'Cantidad':np.sum })

In [13]:
# Tabla con apertura total:
td1 = pd.pivot_table(dfc, index = segmentacion, aggfunc = metricas, fill_value = 0)

# Nombres de columnas:
col_td1 = [

    "Cantidad Total",
    "Consumo Total",
    "Prest. Dist. Total"  ]

# Renombro las columnas:
td1.columns = col_td1


# Tabla con Marca PMO
td8 = funciones.td_consumo(df = dfc, columna = "Marca PMO", segmentacion = segmentacion, metricas = metricas)
print("fin td8 -- " + datetime.now().strftime('%H:%M:%S'))


fin td8 -- 16:30:50


c:\GitHub\Modelos_Costeo\v0.4 - PMO\funciones.py:83: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  td = td.drop(td.columns[col_drop],1)


In [14]:
# Unimos las tablas y exportamos el archivo de consumos:
tdt = pd.concat([td1, td8], axis=1)
tdt.to_csv(resultado_consumos, encoding = 'latin1', decimal=',')

### Cargamos la información del Stock

In [33]:
# Cargamos y cambiamos los nombres de los stocks.
dfs = funciones.leer_xlsx(ruta_stock_zona)
funciones.renombrar_columnas(dfs)

Inicio del proceso: 16:43:00
Se cargó el archivo:  20.1 Stock Mendoza.xlsx a las: 16:44:21
Fin de la carga de archivos


In [34]:
# Levantamos los diferentes archivos auxiliares:

# Rango Edad
Aux_RangoEdad = pd.read_excel(ruta_aux + "Aux_RangoEdad.xlsx")
dfs = pd.merge(left = dfs, right = Aux_RangoEdad[["Edad","Rango_Edad"]], left_on='Edad', right_on='Edad', how = "left")
Aux_RangoEdad = []

# Cartilla Base
Aux_CartillaBase = pd.read_excel(ruta_aux+"Aux_CartillaBase.xlsx")
dfs = pd.merge(left = dfs, right = Aux_CartillaBase[["Plan","Modelo_Cartilla"]], left_on='Plan', right_on='Plan', how = "left")
Aux_CartillaBase = []

# Cobertura Medicamentos del plan
Aux_CobertMedicamentosPlan = pd.read_excel(ruta_aux+"Aux_CobertMedicamentosPlan.xlsx")
dfs = pd.merge(left = dfs, right = Aux_CobertMedicamentosPlan[["Plan","Cobertura_med_id"]], left_on='Plan', right_on='Plan', how = "left")
Aux_CobertMedicamentosPlan = []

# Lista de precios Copagos
Aux_ListaCopagos = pd.read_excel(ruta_aux+"Aux_ListaCopagos.xlsx")
dfs = pd.merge(left = dfs, right = Aux_ListaCopagos[["Plan","Lista_PrecioDesc"]], left_on='Plan', right_on='Plan', how = "left")
Aux_ListaCopagos = []

# Levanto el archivo con los diagnosticos:
dfdiag = pd.read_csv(ruta_diag)
dfs = pd.merge(left = dfs, right = dfdiag[["Persona",'GC', 'Rango_Cons', 'algun_diag', 'prop_diag', 'Marca_Diagnostico']], left_on='Persona', right_on='Persona', how = "left")
dfdiag = []

In [35]:
# Reemplazo valores de filas para tener metricas:
dfs['Diabetes'] = dfs['Diabetes'].replace(['No Diabético', 'Diabético'], [0, 1])
dfs['Hepatitis'] = dfs['Hepatitis'].replace(['Sin Hepatitis', 'Con Hepatitis'], [0, 1])
dfs['PMI'] = dfs['PMI'].replace(['NO', 'SI'], [0, 1])
dfs['Cronico'] = dfs['Cronico'].replace(['NO', 'SI'], [0, 1])
dfs['Oncologico'] = dfs['Oncologico'].replace(['Sin Marca', 'Paciente Oncológico'], [0, 1])
dfs['algun_diag'] = dfs['Marca_Diagnostico'].replace(['Sin Diagnostico', 'Con Diagnostico'], [0, 1])

# Reemplazo los datos faltantes:
dfs["GC"].fillna(0, inplace = True)
dfs["Rango_Cons"].fillna("Muy Bajo", inplace = True)
dfs["algun_diag"].fillna(0, inplace = True)
dfs["prop_diag"].fillna(0, inplace = True)
dfs["Marca_Diagnostico"].fillna("Sin Diagnostico", inplace = True)
dfs['GC'] = dfs['GC'].where(dfs['GC'] == 0 , 1)

# Agregamos los parentescos de los hijos:
condiciones = [
    (dfs['Parentesco IdParentesco'] == 3),
    (dfs['Parentesco IdParentesco'] == 4),
    (dfs['Parentesco IdParentesco'] == 5),
    (dfs['Parentesco IdParentesco'] == 6),
    (dfs['Parentesco IdParentesco'] == 7)]
valores_condic = [1,1,1,1,1]
dfs['Marca Hijo'] = np.select(condiciones, valores_condic, default=0) # se analizaron y surgen de consumo 3 == 0

# Reemplazo valores de filas con replace:
dfs['Sexo'] = dfs['Sexo'].replace(['M', 'F'], [1, 0])
dfs['Discapacidad'] = dfs['Discapacidad'].replace(['N', 'S'], [0, 1])

In [43]:
# reinicio indices de los dataframes de consumo y stock procesados
tdt = tdt.reset_index()

# Unimos el dataframe de stock con el dataframe de consumo:
dff = pd.merge(left = dfs, right = tdt, left_on=("Persona","Periodo"), right_on=("Persona","Periodo_Prest"), how = "left")
dff = dff.fillna(0)

# Exporto a csv el resultado final:
dff.to_csv(resultado_informe, encoding = 'latin1', index = False, decimal=',')


# Tablas que quedaron fuera del estudio:

In [36]:
## SI QUEREMOS EXPORTAR POR PERSONA - PERIODO
# Exportación de archivo sin hacer td:
# dfs.to_csv(archivo_salida, encoding = 'latin1', decimal=',', index = False)


## SI QUEREMOS EXPORTAR POR APERTURAS:
segmentacion = ([ 'Periodo', 'Posicion Asociado ID', 'Plan Agrupado', 'Sexo', 'Rango_Edad','Rango_Cons'])

# Defino las métricas a calcular
metricas = ({   'Stock':np.sum,
                'Edad':np.sum,
                'ACE':np.sum,
                'Oncologico':np.sum,
                'Cronico':np.sum,
                'PMI':np.sum,
                'Hepatitis':np.sum,
                'Diabetes':np.sum,
                'GC':np.sum,
                'algun_diag':np.sum,
                'Marca Hijo':np.sum,
            })


td = pd.pivot_table(dfs, index = segmentacion, aggfunc = metricas)
base_asociados = pd.DataFrame(td)
# Exporto la información en csv:
##### base_asociados.to_csv(archivo_salida , encoding = 'latin1', decimal=',', index = False)

# dfcp = pd.read_csv('./PerPrest-Persona-Consumo-Bnknn_v3.csv', encoding = 'latin1', decimal=',')
# dff = pd.merge(left = dfs, right = dfcp, left_on=("Persona","Periodo"), right_on=("Persona","Periodo_Prest"), how = "left")
# dff = dff.fillna(0)

In [37]:
base_asociados.head()

ACE  \
Periodo Posicion Asociado ID Plan Agrupado Sexo Rango_Edad    Rango_Cons         
202010  23                   AMARI         F    Entre 0 y 1   Bajo        4.41   
                                                              Medio       7.45   
                                                              Muy Bajo    2.94   
                                                Entre 11 y 15 Bajo        3.06   
                                                              Medio       3.57   

                                                                          Cronico  \
Periodo Posicion Asociado ID Plan Agrupado Sexo Rango_Edad    Rango_Cons            
202010  23                   AMARI         F    Entre 0 y 1   Bajo              0   
                                                              Medio             0   
                                                              Muy Bajo          0   
                                                Entre 11 y 15 Bajo              0   
                                                              Medio             0   

                                                                          Diabetes  \
Periodo Posicion Asociado ID Plan Agrupado Sexo Rango_Edad    Rango_Cons             
202010  23                   AMARI         F    Entre 0 y 1   Bajo               0   
                                                              Medio              0   
                                                              Muy Bajo           0   
                                                Entre 11 y 15 Bajo               0   
                                                              Medio              0   

                                                                          Edad  \
Periodo Posicion Asociado ID Plan Agrupado Sexo Rango_Edad    Rango_Cons         
202010  23                   AMARI         F    Entre 0 y 1   Bajo           1   
                                                              Medio          3   
                                                              Muy Bajo       1   
                                                Entre 11 y 15 Bajo          76   
                                                              Medio         88   

                                                                          GC  \
Periodo Posicion Asociado ID Plan Agrupado Sexo Rango_Edad    Rango_Cons       
202010  23                   AMARI         F    Entre 0 y 1   Bajo         0   
                                                              Medio        0   
                                                              Muy Bajo     0   
                                                Entre 11 y 15 Bajo         0   
                                                              Medio        0   

                                                                          Hepatitis  \
Periodo Posicion Asociado ID Plan Agrupado Sexo Rango_Edad    Rango_Cons              
202010  23                   AMARI         F    Entre 0 y 1   Bajo                0   
                                                              Medio               0   
                                                              Muy Bajo            0   
                                                Entre 11 y 15 Bajo                0   
                                                              Medio               0   

                                                                          Marca Hijo  \
Periodo Posicion Asociado ID Plan Agrupado Sexo Rango_Edad    Rango_Cons               
202010  23                   AMARI         F    Entre 0 y 1   Bajo                 3   
                                                              Medio                5   
                                                              Muy Bajo             2   
                                                Entre 11 y 15 Bajo                 6   
                           

# Aperturas que quedaron fuera del estudio

In [ ]:
# CREAMOS LA COLUMNA: "Tipo Int. ID"

dfint = pd.read_excel(ruta_int)
dfc = pd.merge( left = dfc, right = dfint[['Paciente ID','Orden Int. ID','Tipo Int. ID']], 
               right_on=['Paciente ID','Orden Int. ID'], 
               left_on=['Persona','Orden Relacionada ID'], how = "left")
del dfint
dfc['Tipo Int. ID'] = dfc['Tipo Int. ID'].replace(np.nan,"Otras Int") # Reemplazamos los falta dato por "otras Int"

condiciones_dfint = [
    (dfc['Origen Prestacion ID'] == 'I')]
valores_dfint = [dfc['Tipo Int. ID']]   
dfc['Tipo Int. ID'] = np.select(condiciones_dfint, valores_dfint, default = 'No Considerar')

In [ ]:
# APERTURA TOTAL

condiciones2 = [
    (dfc['Origen Facturacion ID'] == 'A') ,
    (dfc['Origen Facturacion ID'] == 'C') ,
    (dfc['Nomenclador ID'] == 'ME') ,
    (dfc['Nomenclador ID'] == 'M1') ,
    (dfc['Origen Prestacion ID'] == 'I')]

# Lista de resultados en función de las selecciones
valores_condic2 = ['Naj', 'NC', 'MEd', 'MEd', 'Internacion sin A,C,Med']     
dfc['Apertura Total'] = np.select(condiciones2, valores_condic2, default = 'Ambulatorio sin A,C,Med') # Generación de la columna calculada

In [ ]:
# CREAMOS LA COLUMNA: "Amb por Rubros"

condiciones3 = [
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'A'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'I'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'C'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'D'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'E'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'F'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'G'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'N'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'M'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'J'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'R'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'S'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'T'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'U'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'V'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'W'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'X'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'M1'),

    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'NP'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'DI'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'NC'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Nomenclador ID'] == 'NA'),
    
    (dfc['Origen Prestacion ID'] == 'A') ]

# Lista de resultados en función de las selecciones
valores_condic3 = ['Consulta', 'Laboratorio', 'Imagenes', 'Imagenes', 'Imagenes', 
                  'Imagenes', 'Imagenes', 'Fisiokinesio', 'Salud Mental', 
                  'Odontologia', 'Odontologia', 'Odontologia', 'Odontologia', 
                   'Odontologia', 'Odontologia', 'Odontologia', 'Odontologia',
                   'Amb ME','Amb ME','Amb NP','Amb DI', 'Amb NC',
                   'Amb NA','Resto Amb']   

dfc['Amb por Rubros'] = np.select(condiciones3, valores_condic3, default = 'No Considerar') # Generación de la columna calculada

In [ ]:
# CREAMOS LA COLUMNA: "Medicamentos por Origen"

condiciones4 = [
    
    # Condiciones para marcar medicamentos en internacion:
    (dfc['Origen Prestacion ID'] == 'I') & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Origen Prestacion ID'] == 'I') & (dfc['Nomenclador ID'] == 'M1'),
    
    # Condiciones para marcar medicamentos en amb -> no se filtra origen (por defecto mandar a AMB)
    (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Nomenclador ID'] == 'M1') ]

# Lista de resultados en función de las selecciones
valores_condic4 = ['Med. Int', 'Med. Int', 'Med. Amb', 'Med. Amb']     

# Generación de la columna calculada
dfc['Medicamentos por Origen'] = np.select(condiciones4, valores_condic4, default = 'No Considerar')

In [ ]:
# CREAMOS LA COLUMNA: "Medicamentos por via"

condiciones5 = [
    
    # Condiciones para marcar medicamentos en internacion:
    (dfc['Provision Acreedor ID'] == 6) & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Provision Acreedor ID'] == 6) & (dfc['Nomenclador ID'] == 'M1'),

    (dfc['Provision Acreedor ID'] == 66) & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Provision Acreedor ID'] == 66) & (dfc['Nomenclador ID'] == 'M1'),
    
    (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Nomenclador ID'] == 'M1') ]

# Lista de resultados en función de las selecciones
valores_condic5 = ['Farmacia', 'Farmacia', 'Provision', 'Provision', 'Otras vias', 'Otras vias']     

# Generación de la columna calculada
dfc['Medicamentos por Via'] = np.select(condiciones5, valores_condic5, default = 'No Considerar')

In [ ]:
# CREAMOS LA COLUMNA: "Medicamentos por cobertura"
dfmed = pd.read_excel(medicamentos_especiales)

# Cruzamos con el dfc:
dfc = pd.merge( left = dfc, right = dfmed[["Prestacion","Tipo_med"]], 
               left_on='Prestacion', right_on='Prestacion', how = "left")

# Libero memoria:
dfmed = []

# Cálculos auxiliares para definir % cobertura:
# Creo el cálculo auxiliar con *10 para quedarnos con la decena de la cobertura
dfc['Cob_prec_sug'] = (dfc.Consumo / dfc.Precio_sug)*10
# Reemplazo valores +inf y -inf por 0
dfc['Cob_prec_sug'] = dfc['Cob_prec_sug'].replace([np.inf, -np.inf], 0)
# Redondeo a 0 y multiplico por 10 para quedarme con la decena
dfc['Cob_prec_sug'] = round(dfc['Cob_prec_sug'],0)*10


# Coberturas
dfmed2 = pd.read_excel(cobertura_medicamentos)

# Cruzamos con el dfc:
dfc = pd.merge( left = dfc, right = dfmed2[["Plan","Cobertura_med_id"]], 
               left_on='Plan', right_on='Plan', how = "left")

# Libero memoria:
dfmed2 = []

# MARCA COBERTURA BASE
# Transformo las provisiones en función de su descripción
condiciones_c = [
    (dfc.Cobertura_med_id == 301) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 302) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 302) & (dfc.Cobertura_DW == 50) & (dfc.Cob_prec_sug == 50),    
    (dfc.Cobertura_med_id == 303) & (dfc.Cobertura_DW == 50) & (dfc.Cob_prec_sug == 50),
    (dfc.Cobertura_med_id == 304) & (dfc.Cobertura_DW == 60) & (dfc.Cob_prec_sug == 60),
    (dfc.Cobertura_med_id == 305) & (dfc.Cobertura_DW == 70) & (dfc.Cob_prec_sug == 70),
    (dfc.Cobertura_med_id == 306) & (dfc.Cobertura_DW == 80) & (dfc.Cob_prec_sug == 80),
    (dfc.Cobertura_med_id == 307) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 307) & (dfc.Cobertura_DW == 60) & (dfc.Cob_prec_sug == 60),
    (dfc.Cobertura_med_id == 308) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 310) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 312) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 312) & (dfc.Cobertura_DW == 60) & (dfc.Cob_prec_sug == 60),
    (dfc.Cobertura_med_id == 313) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 313) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40),
    (dfc.Cobertura_med_id == 315) & (dfc.Cobertura_DW == 40) & (dfc.Cob_prec_sug == 40) 
]

# Lista de resultados en función de las selecciones
valores_condic_c = ['Cob. Base 40%', 
                  'Cob. Base 40%',
                  'Cob. Base 50%',
                  'Cob. Base 50%',
                  'Cob. Base 60%',
                  'Cob. Base 70%',
                  'Cob. Base 80%',
                  'Cob. Base 40%',
                  'Cob. Base 60%',
                  'Cob. Base 40%',
                  'Cob. Base 40%',
                  'Cob. Base 40%',
                  'Cob. Base 60%',
                  'Cob. Base 40%',
                  'Cob. Base 40%',
                  'Cob. Base 40%']     

# Generación de la columna calculada
dfc['Marca_Cob_Base'] = np.select(condiciones_c, valores_condic_c, default = 'No Cob. Base')

# MARCA COBERTURA 100%
condiciones = [(dfc.Cobertura_DW == 100) & (dfc.Cob_prec_sug == 100)]
valores_condic = ['Cobertura 100%']  # Lista de resultados en función de las selecciones

# Generación de la columna calculada
dfc['Marca_Cobertura_100'] = np.select(condiciones, valores_condic, default = 'No Cobertura 100%')

# Genero la agrupación final med
condiciones = [
    # Tipo medicamento es MACBI, ME6000 o M1 o R310.
    dfc.Tipo_med == 'ME6000000 y ME1600085',
    dfc.Tipo_med.isnull() == 0,
    dfc.Marca_Cob_Base != 'No Cob. Base',
    dfc.Marca_Cobertura_100 == "Cobertura 100%",
    dfc["Tipo Orden ID"] == 4,
    dfc["Nomenclador ID"] == 'ME',
    dfc["Nomenclador ID"] == 'M1']
    
# Lista de resultados en función de las selecciones
valores_condic = ["Prest Genéricas", dfc.Tipo_med, dfc.Marca_Cob_Base, dfc.Marca_Cobertura_100, "Form. 4", "Otros", "Otros"]  

dfc['Medicamentos por Cobertura'] = np.select(condiciones, valores_condic, default = 'No Considerar') # Generación de la columna calculada

In [ ]:
## SEGMENTACIONES Y METRICAS POSIBLES ANTES DE EXPORTAR:

# Segmentaciones y métricas comúnes:
segmentacion = (['Periodo_Prest', 'Persona'])

# dejo para probar procesamientos más rápido (debugging)
segmentacion2 = ['Periodo', 'Sexo']


# Defino las métricas a calcular
metricas = ({'Consumo':[np.sum], 'Prestacion':pd.Series.nunique,'Cantidad':[np.sum]})


metricas2 = ({  'Consumo':[np.sum,len],
                'Cantidad':np.sum })

In [ ]:
# CREAMOS LA TABLA PMO:

# Marca PMO
td8 = funciones.td_consumo(df = dfc, columna = "Marca PMO", segmentacion = segmentacion, metricas = metricas)
print("fin td8 -- " + datetime.now().strftime('%H:%M:%S'))

In [ ]:
# CREAMOS LAS DIFERENTES TABLAS:

td1 = pd.pivot_table(dfc, index = segmentacion, aggfunc = metricas, fill_value = 0)

# Nombres de columnas:
col_td1 = [

    "Cantidad Total",
    "Consumo Total",
    "Prest. Dist. Total"  ]

# Renombro las columnas:
td1.columns = col_td1


# Apertura total
td2 = funciones.td_consumo(df = dfc, columna = "Apertura Total", segmentacion = segmentacion, metricas = metricas)
print("fin td2 -- " + datetime.now().strftime('%H:%M:%S'))

# Amb por rubro
td3 = funciones.td_consumo(df = dfc, columna = "Amb por Rubros", segmentacion = segmentacion, metricas = metricas)
print("fin td3 -- " + datetime.now().strftime('%H:%M:%S'))

# Medicamentos por origen
td4 = funciones.td_consumo(df = dfc, columna = "Medicamentos por Origen", segmentacion = segmentacion, metricas = metricas)
print("fin td4 -- " + datetime.now().strftime('%H:%M:%S'))

# Medicamentos por Via
td5= funciones.td_consumo(df = dfc, columna = "Medicamentos por Via", segmentacion = segmentacion, metricas = metricas)
print("fin td5 -- " + datetime.now().strftime('%H:%M:%S'))

# Medicamentos por Cobertura
td6 = funciones.td_consumo(df = dfc, columna = "Medicamentos por Cobertura", segmentacion = segmentacion, metricas = metricas)
print("fin td6 -- " + datetime.now().strftime('%H:%M:%S'))

# Internaciones por tipo
td7 = funciones.td_consumo(df = dfc, columna = "Tipo Int. ID", segmentacion = segmentacion, metricas = metricas)
print("fin td7 -- " + datetime.now().strftime('%H:%M:%S'))

# Marca PMO
td8 = funciones.td_consumo(df = dfc, columna = "Marca PMO", segmentacion = segmentacion, metricas = metricas)
print("fin td8 -- " + datetime.now().strftime('%H:%M:%S'))

In [ ]:
tdt = pd.concat([td1, td2, td3, td4, td5, td6, td7, td8], axis=1)
#tdt.to_csv(archivo_exportacion, encoding = 'latin1', decimal=',')